# Stabilizatión of the simulation

In [2]:
## -----Import essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Define important variables

temp = 9.5
pressure = 0.1

#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd("T_9.5_0.1_ramp.gsd",frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt=0.005);

#------Define group

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 5.0, tauP = 1.5, P = pressure);

#-----Write output and Run the Simulation

log_file = "T_" + str(temp) + "_" + str(pressure) + "_stabilization.log"
gsd_file = "T_" + str(temp) + "_" + str(pressure) + "_stabilization.gsd" 
meso_gsd_file = "T_CM_" + str(temp) + "_" + str(pressure) + "_stabilization.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

notice(2): Group "all" created containing 9000 particles
notice(2): constrain.rigid(): Removing all particles part of rigid bodies (except central particles).Particle tags may change.
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "rigid_center" created containing 1000 particles


In [ ]:
hoomd.run(3e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 2224521 / 2524000 | TPS 51.898 | ETA 01:36:10
Time 00:00:20 | Step 2225100 / 2524000 | TPS 57.8906 | ETA 01:26:03
Time 00:00:30 | Step 2225713 / 2524000 | TPS 61.24 | ETA 01:21:10
Time 00:00:40 | Step 2226343 / 2524000 | TPS 62.8965 | ETA 01:18:52
Time 00:00:50 | Step 2227001 / 2524000 | TPS 65.6323 | ETA 01:15:25
Time 00:01:00 | Step 2227652 / 2524000 | TPS 65.0819 | ETA 01:15:53
Time 00:01:10 | Step 2228272 / 2524000 | TPS 61.9899 | ETA 01:19:30
Time 00:01:20 | Step 2228859 / 2524000 | TPS 58.6738 | ETA 01:23:50
Time 00:01:30 | Step 2229425 / 2524000 | TPS 56.565 | ETA 01:26:47
Time 00:01:40 | Step 2229985 / 2524000 | TPS 55.8527 | ETA 01:27:44
Time 00:01:50 | Step 2230551 / 2524000 | TPS 56.537 | ETA 01:26:30
Time 00:02:00 